# 📘 RAG-Based QA from Predefined Google Drive PDFs (FAISS + Groq + Streamlit)


Cell 1: Install Dependencies

In [16]:
!pip install requests PyPDF2 streamlit groq langchain langchain-community faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


Cell 2:  RAG App Code

In [19]:
%%writefile app.py

# Full Application Code
import os
import requests
from io import BytesIO
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from groq import Groq
import streamlit as st

# ✅ Set your Groq API Key here
os.environ["GROQ_API_KEY"] = [YOUR GROQ API KEY]  # Replace with your actual API key

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# 📚 Predefined Google Drive PDF Links
drive_links = [
    "https://drive.google.com/file/d/1JPf0XvDhn8QoDOlZDrxCOpu4WzKFESNz/view?usp=sharing"]

# 📥 Download PDF from Google Drive
def download_pdf_from_drive(drive_link):
    try:
        file_id = drive_link.split("/d/")[1].split("/")[0]
        url = f"https://drive.google.com/uc?id={file_id}&export=download"
        response = requests.get(url)
        response.raise_for_status()
        return BytesIO(response.content)
    except Exception as e:
        raise Exception(f"Failed to download PDF: {e}")

# 📖 Extract text from PDF
def extract_text_from_pdf(pdf_stream):
    reader = PdfReader(pdf_stream)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# ✂️ Chunk text
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

# 🔗 Create embeddings and store in FAISS
def create_embeddings_and_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_db

# 💬 Query the FAISS DB with Groq

def query_vector_db(query, vector_db):
    docs = vector_db.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in docs])
    completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"Use the following context:\n{context}"},
            {"role": "user", "content": query}
        ],
        model="llama3-8b-8192"
    )
    return completion.choices[0].message.content

# 🚀 Streamlit App
st.title("📄 RAG App: Google Drive PDF QA")
all_chunks = []

for link in drive_links:
    try:
        st.write(f"🔗 Processing: {link}")
        pdf_stream = download_pdf_from_drive(link)
        text = extract_text_from_pdf(pdf_stream)
        chunks = chunk_text(text)
        all_chunks.extend(chunks)
        st.success(f"✅ Extracted {len(chunks)} chunks.")
    except Exception as e:
        st.error(str(e))

if all_chunks:
    vector_db = create_embeddings_and_store(all_chunks)
    st.success("🧠 Embeddings stored in FAISS.")

    user_query = st.text_input("🔍 Enter your question:")
    if user_query:
        answer = query_vector_db(user_query, vector_db)
        st.markdown("### 💬 Answer")
        st.write(answer)


Overwriting app.py


Cell 3: Run Streamlit App in Colab

In [ ]:
 !wget -q -O - ipv4.icanhazip.com

!streamlit run app.py & npx localtunnel --port 8501


34.82.92.215


⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.92.215:8501

⠦⠧⠇⠏⠋⠙your url is: https://breezy-otter-40.loca.lt
/content/app.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2025-07-17 06:42:27.453045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752734547.484471   14310 cuda_dnn.cc:831